In [46]:
from edgar_prelim import *
from edgar_query import *
from edgar_title import *
import re
from bs4 import BeautifulSoup
import pandas as pd
%load_ext autoreload
%autoreload 2
%matplotlib inline
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
BAC='0000070858'
C='0000831001'
STI = '0000750556'
JPM = '0000019617'
WFC = '0000072971'
GS = '0000886982'
COF = '0000927628'
MS = '0000895421'
USB = '0000036104'
PNC = '0000713676'
BK='0001390777'
SCHW='0000316709'
STT='0000093751'
BBT='0000092230'
UBSH='0000883948'

In [ ]:
d=date(2012, 1, 20)
df = extract_prelim_statements('0001403475', start=d, end=d)
df

In [49]:
d = date(2019, 1, 15)
filings = query_edgar_for_filings(JPM, "8-K", start=d, end=d, require_xbrl=False)
report = query_edgar_for_submission_text(filings[1].href)
submission = load_submission(report.href)
tables = list(parse_tables(submission))
items_from_tables(tables)

,item,fiscal_period,item_rank,item_value,src_column,src_row,src_table,src_units,src_value,table_rank
0,book value per share,2018Q4,0,7.035000e+01,QUARTERLY TRENDS 4Q18,Book value per share,CONSOLIDATED FINANCIAL HIGHLIGHTS,"(in millions, except per share and ratio data)",70.35,4
1,interest income,2018Q4,0,2.103800e+10,QUARTERLY TRENDS 4Q18,Interest income,CONSOLIDATED STATEMENTS OF INCOME,"(in millions, except per share and ratio data)",21038,0
2,net income,2018Q4,0,7.066000e+09,QUARTERLY TRENDS 4Q18,NET INCOME,CONSOLIDATED STATEMENTS OF INCOME,"(in millions, except per share and ratio data)",$7066,0
3,net interest income,2018Q4,0,1.435400e+10,QUARTERLY TRENDS 4Q18,Net interest income,CONSOLIDATED STATEMENTS OF INCOME,"(in millions, except per share and ratio data)",14354,0
4,provision for loan losses,2018Q4,0,1.548000e+09,QUARTERLY TRENDS 4Q18,Provision for credit losses,CONSOLIDATED STATEMENTS OF INCOME,"(in millions, except per share and ratio data)",1548,0
5,total revenue,2018Q4,0,2.610900e+10,QUARTERLY TRENDS 4Q18,TOTAL NET REVENUE,CONSOLIDATED STATEMENTS OF INCOME,"(in millions, except per share and ratio data)",26109,0


In [7]:
bs = BeautifulSoup(submission.documents[-1].text, features='lxml')

In [8]:
tables = bs.find_all('table')

In [32]:
re.search(        fr'^{UNAUDITED}(?:(?:quarterly|condensed|select(?:ed)?)\s+)*consolidated\s+'
        fr'(?:(?:select(?:ed)?|key)\s+)*'
        r'(?:financial|operating|earnings|capital)?(?:\s+and\s+other)?'
        r'(?:\s+(?:results|metrics|highlights?|summary|information|data))+', 'CONSOLIDATED FINANCIAL HIGHLIGHTS'.lower())

<re.Match object; span=(0, 33), match='consolidated financial highlights'>

In [47]:
    table_matches = find_tag_text_matches(
        [tables[2]],
        title_inclusion_patterns,,
        max_tables=2)


In [48]:
list(table_matches)

TypeError: unhashable type: 'list'

In [5]:
[t.title for t in tables]

['CONSOLIDATED STATEMENTS OF INCOME',
 'CONSOLIDATED STATEMENTS OF INCOME',
 'CONSOLIDATED FINANCIAL HIGHLIGHTS, CONTINUED',
 'CONSOLIDATED BALANCE SHEETS',
 'CONSOLIDATED BALANCE SHEETS',
 'CONSOLIDATED BALANCE SHEETS',
 'FINANCIAL HIGHLIGHTS',
 'FINANCIAL HIGHLIGHTS',
 'FINANCIAL HIGHLIGHTS',
 'FINANCIAL HIGHLIGHTS',
 'FINANCIAL HIGHLIGHTS',
 'FINANCIAL HIGHLIGHTS',
 'FINANCIAL HIGHLIGHTS',
 'FINANCIAL HIGHLIGHTS',
 'FINANCIAL HIGHLIGHTS',
 'FINANCIAL HIGHLIGHTS',
 'FINANCIAL HIGHLIGHTS',
 'FINANCIAL HIGHLIGHTS',
 'FINANCIAL HIGHLIGHTS, CONTINUED',
 'FINANCIAL HIGHLIGHTS, CONTINUED',
 'FINANCIAL HIGHLIGHTS, CONTINUED',
 'FINANCIAL HIGHLIGHTS, CONTINUED',
 'FINANCIAL HIGHLIGHTS, CONTINUED',
 'FINANCIAL HIGHLIGHTS, CONTINUED',
 'FINANCIAL HIGHLIGHTS, CONTINUED',
 'FINANCIAL HIGHLIGHTS, CONTINUED',
 'FINANCIAL HIGHLIGHTS, CONTINUED',
 'FINANCIAL HIGHLIGHTS, CONTINUED',
 'FINANCIAL HIGHLIGHTS, CONTINUED',
 'FINANCIAL HIGHLIGHTS, CONTINUED',
 'FINANCIAL HIGHLIGHTS, CONTINUED',
 'FINANCIAL

In [37]:
units_from_table(tables[0])

("($000's omitted)", 1000.0)

In [50]:
re.search(prelim_items[-2].name_pattern, 'Book value per common share  GAAP (a)(b)', flags=re.IGNORECASE)

<re.Match object; span=(0, 41), match='Book value per common share \x96 GAAP (a)(b)'>

In [49]:
prelim_items[-2].name_pattern

'^(?:total\\s+)?(?:common\\s+)?(?:(?:shareholders\\W?\\s+)?equity\\s+)?\\(?book\\s+value\\)?(?:\\s+per\\s+(?:common\\s+)?share(?:\\s+of\\s+common\\s+stock)?)?,?(?:(?:\\s+at)?\\s+\\(?period\\s*-?\\s*end\\)?)?(?:\\s*[—\\x97\\x96–-]*\\s*gaap)?(?:\\s*(?:\\(\\w{1,2}\\)|\\*))*$'

In [40]:
re.search(r'([1-4])\s*q(?:tr)?\s*((?:20)?\s*\d\s*\d)', 'QUARTERLY TRENDS 3Q11'.lower())

<re.Match object; span=(17, 21), match='3q11'>

In [ ]:
%%prun
tables = list(parse_tables(submission))
items_from_tables(tables)

In [7]:
bs = BeautifulSoup(submission.documents[-1].text, features='lxml')

In [8]:
tables = bs.find_all('table')

In [12]:
for t in tables:
    title = title_from_table_tag(t)
    if title:
        print(title[0])

Consolidated Statement of Income
CITIGROUP  FINANCIAL SUMMARY
CITIGROUP  FINANCIAL SUMMARY
CITIGROUP CONSOLIDATED STATEMENT OF INCOME
CITIGROUP CONSOLIDATED STATEMENT OF INCOME
CITIGROUP CONSOLIDATED BALANCE SHEET
CITIGROUP CONSOLIDATED BALANCE SHEET
CITIGROUP CONSOLIDATED BALANCE SHEET
CITIGROUP CONSOLIDATED BALANCE SHEET


In [ ]:
for t in tables:
   print(t.df[0])

In [134]:
item_value_from_table(tables[4].df.iloc[:, [0, 1]], prelim_items[-1].name_pattern, 1)

ItemValue(src_row='Book value per common share', src_column='ThreeMonthsEnded June30 2007', src_value='$48.33', value=48.33)

In [135]:
max_fiscal_period(tables[4].df)

(2, '2030Q2')

In [134]:
discard_prelim(UBSH, date(2018, 11, 16))

In [60]:
force_reload_prelim('0001005817',  date(2013, 4, 26))

2019-05-15 09:57:01,569 edgar_prelim INFO MainThread Loading preliminary statements for 0001005817.
2019-05-15 09:57:01,569 edgar_prelim INFO MainThread Deleting old preliminary statements for 0001005817.


In [33]:
force_reload_prelim('0001411974',  date(2009, 4, 28))

2019-04-29 17:08:53,639 edgar_prelim INFO MainThread Loading preliminary statements for 0001487052.
2019-04-29 17:08:53,641 edgar_prelim INFO MainThread Deleting old preliminary statements for 0001487052.
2019-04-29 17:08:57,335 edgar_prelim INFO MainThread Loading filing: Filing(date=datetime.date(2013, 2, 7), type='8-K', href='https://www.sec.gov/Archives/edgar/data/1487052/000148705213000003/0001487052-13-000003-index.htm').
2019-04-29 17:09:04,335 edgar_prelim INFO MainThread Loading filing: Filing(date=datetime.date(2013, 4, 24), type='8-K', href='https://www.sec.gov/Archives/edgar/data/1487052/000148705213000022/0001487052-13-000022-index.htm').


In [358]:
d=date(2010, 4, 21)
filings = query_edgar_for_filings(STI, "8-K", start=d, end=d, require_xbrl=False)
report = query_edgar_for_submission_text(filings[0].href)
submission = load_submission(report.href)

In [490]:
table_tags = BeautifulSoup(submission.documents[1].text, features='lxml').find_all('table')

In [ ]:
title_from_table_tag(table_tags[18])

In [153]:
table_tags = list(filter(lambda x: title_from_table_tag(x) is not None, table_tags))

In [ ]:
pd.read_html(table_tags[0].prettify(), flavor='html5lib')[0].pipe(drop_single_value_rows) \
    .pipe(drop_null_rows_and_columns) \
    .fillna(value='') \
    .pipe(collapse_identical_rows) \
    .pipe(collapse_identical_columns) \
    .pipe(drop_bogus_header_rows)  \
    .pipe(collapse_header_rows) \
    .pipe(collapse_columns_with_identical_header, start_col=0)

In [223]:
parse_fiscal_period('QTD 2011')

'2011QTD'

In [192]:
pattern = (
    r'^(?:unaudited\s+)?(?:(?:selected|key)\s+)?(?:quarterly\s+)?'
    r'(?:financial|operating|income\s+statement)\s+'
    fr'(?:highlights?|information|results|data)$'
)
re.search(pattern, 'FINANCIAL HIGHLIGHTS', flags=re.IGNORECASE)

<re.Match object; span=(0, 20), match='FINANCIAL HIGHLIGHTS'>

In [123]:
parse_value('$48.33 	')

48.33

In [69]:
re.search('\w', 'Interest and dividend income')

<re.Match object; span=(0, 1), match='I'>

In [57]:
widget = query_prelims_for_override('0001005817', filing_date='2016-01-28', item=None)
widget

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

In [58]:
widget.save()

,filing_date,item,cik,filing_href,filing_type,fpe_date,fiscal_year,fiscal_quarter,fiscal_period,item_value
0,2016-01-28,net income,0001005817,Unknown,8-K,None,None,None,None,None
1,2016-01-28,net interest income,0001005817,Unknown,8-K,None,None,None,None,None
2,2016-01-28,interest income,0001005817,Unknown,8-K,None,None,None,None,None
3,2016-01-28,provision for loan losses,0001005817,Unknown,8-K,None,None,None,None,None
4,2016-01-28,total revenue,0001005817,Unknown,8-K,None,None,None,None,None
5,2016-01-28,book value per share,0001005817,Unknown,8-K,None,None,None,None,None


In [8]:
cik_df = pd.read_sql("select * from cik", prelim_engine)

In [9]:
cik_df.head()

,cik,sic,sic_description,company_name,ticker
0,0000004962,6199,FINANCE SERVICES,AMERICAN EXPRESS CO,AXP
1,0000007789,6022,STATE COMMERCIAL BANKS,ASSOCIATED BANC-CORP,ASB
2,0000016614,6036,"SAVINGS INSTITUTIONS, NOT FEDERALLY CHARTERED",CAMCO FINANCIAL CORP,CAFI
3,0000018349,6021,NATIONAL COMMERCIAL BANKS,SYNOVUS FINANCIAL CORP,SNV
4,0000019612,6022,STATE COMMERCIAL BANKS,CHEMICAL FINANCIAL CORP,CHFC


In [10]:
cik_df.to_csv("../src/resources/cik.csv")

In [6]:
%pwd

'C:\\Users\\swido\\Documents\\dev\\edgar_prelim\\notebooks'